In [1]:
import os
import io
import sys
from dotenv import load_dotenv
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(
    api_key = anthropic_api_key
)

In [12]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [14]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [3]:
python_code = """
def add(a, b):
    return a + b
"""

In [7]:
exec(python_code)

result = add(3, 5)
print(result)

8


In [8]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [9]:
claude = anthropic.Anthropic()
CLAUDE_MODEL = "claude-3-haiku-20240307"  # Cheapest model

In [ ]:
# Generationg output

def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [15]:
optimize_claude(python_code)

#include <iostream>
#include <cmath>
#include <limits>

long long add(long long a, long long b) {
    return a + b;
}

int main() {
    long long a = 1000000000000;
    long long b = 2000000000000;
    std::cout << add(a, b) << std::endl;
    return 0;
}

In [ ]:
# Streaming output

def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )

    with result as stream:
        for text in stream.text_stream:
            yield text.replace('```cpp\n','').replace('```','')


In [22]:
for chunk in stream_claude(python_code):
    print(chunk, end='', flush=True)


cpp
#include <iostream>
#include <limits>
#include <cmath>
#include <iomanip>

long long add(long long a, long long b) {
    return a + b;
}

int main() {
    long long a = std::numeric_limits<long long>::max() - 100;
    long long b = 200;
    std::cout << std::setprecision(0) << std::fixed << add(a, b) << std::endl;
    return 0;
}
